In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [16]:
import cv2
from imutils import paths

In [7]:
tensorflow.__version__

'2.2.0'

In [8]:
import tensorflow

In [12]:
paths = os.listdir('data')[1:]

In [14]:
os.path.join('.','with_mask')

'./with_mask'

In [17]:
dataset_path = './data'
lst = os.listdir(dataset_path)
if '.DS_Store' in lst:
    lst.remove('.DS_Store')
data = []
labels = []
paths_list = list(paths.list_images(dataset_path))

In [18]:
paths_list

['./data/with_mask/augmented_image_218.jpg',
 './data/with_mask/augmented_image_19.jpg',
 './data/with_mask/augmented_image_31.jpg',
 './data/with_mask/augmented_image_25.jpg',
 './data/with_mask/augmented_image_230.jpg',
 './data/with_mask/419-with-mask.jpg',
 './data/with_mask/330-with-mask.jpg',
 './data/with_mask/260-with-mask.jpg',
 './data/with_mask/27-with-mask.jpg',
 './data/with_mask/327-with-mask.jpg',
 './data/with_mask/277-with-mask.jpg',
 './data/with_mask/30-with-mask.jpg',
 './data/with_mask/augmented_image_3.jpg',
 './data/with_mask/augmented_image_152.jpg',
 './data/with_mask/augmented_image_146.jpg',
 './data/with_mask/309-with-mask.jpg',
 './data/with_mask/259-with-mask.jpg',
 './data/with_mask/420-with-mask.jpg',
 './data/with_mask/augmented_image_191.jpg',
 './data/with_mask/155-with-mask.jpg',
 './data/with_mask/augmented_image_185.jpg',
 './data/with_mask/89-with-mask.jpg',
 './data/with_mask/273-with-mask.jpg',
 './data/with_mask/323-with-mask.jpg',
 './data/wit

In [ ]:
for path in paths_list:
    labelpath = os.path.join('.',path)
    label = labelpath.split('/')[-2]
    image = cv2.imread()

In [19]:
for i in paths_list:
    
    
    label = i.split('/')[-2]
    
    image = cv2.imread(i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    
    data.append(image)
    
    labels.append(label)

In [20]:
len(data)

1376

In [21]:
len(labels)

1376

In [22]:
data = np.array(data)/255.0
labels = np.array(labels)

In [23]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [24]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [26]:
mygen = ImageDataGenerator(
    horizontal_flip=True,
    fill_mode="nearest")

In [27]:
base = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

9412608/9406464 [==============================] - 2s 0us/step


In [28]:
top = base.output
top = MaxPooling2D(pool_size=(4, 4))(top)
top = Flatten(name="flatten")(top)
top = Dense(128, activation="relu")(top)
top = Dense(64, activation="relu")(top)
top = Dropout(0.5)(top)
top = Dense(2, activation="softmax")(top)

In [29]:
model = Model(inputs = base.input, outputs=top)

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

In [31]:
int_lr = 1e-3
epoch = 25
bs= 8

In [32]:
adm = Adam(lr=int_lr, decay=int_lr / epoch)

In [33]:
model.compile(loss="binary_crossentropy", optimizer=adm,metrics=["accuracy"])

In [35]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [36]:
fit_model = model.fit_generator(mygen.flow(trainX, trainY, batch_size=bs),
                        steps_per_epoch=len(trainX) // bs,
                        validation_data=(testX, testY),
                        validation_steps=len(testX) // bs,
                        epochs=epoch)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
 28/137 [=====>........................] - ETA: 1:29 - loss: 0.8457 - accuracy: 0.7812

KeyboardInterrupt: 